# Assessing metadata found in the Zenodo repository for datasets

**Aim.** The aim of this experiment is to evaluate if the RDF metadata published by [Zenodo](https://zenodo.org/) is following the FAIR principles guideline.

**Material and methods.** In this experiment, we launch a set of FAIR tests which are mostly based on SPARQL requests.

**Results.** We produce an UpSetPlot summarizing the number of resource validating each test, allowing to show some validation combination profiles.

In [1]:
import sys
parentdir = "../.."
sys.path.insert(0, parentdir)

import requests
from os import path
from tqdm.notebook import tqdm
import pandas as pd
import time

from metrics.WebResource import WebResource
from metrics.FAIRMetricsFactory import FAIRMetricsFactory
from metrics.AbstractFAIRMetrics import AbstractFAIRMetrics

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST driver version for 101.0.4951
[WDM] - Driver [/home/trosnet/.wdm/drivers/chromedriver/linux64/101.0.4951.41/chromedriver] found in cache


In [2]:
from rdflib import ConjunctiveGraph, Namespace, URIRef
from rdflib.namespace import RDF, RDFS

schema = Namespace("http://schema.org/")

## 1. Retrieve a list of DOIs from the Zenodo repository API

In [3]:
TIMEOUT = (10, 300)
#NB = 10000
NB = 5000
TYPE = "dataset"
OUTPUT_DIR = "../results/zenodo_data/"

def zenodoRestRequest():
    print("REST request to zenodo...")
    # rest request
    url = 'https://zenodo.org/api/records/?sort=mostrecent&page=1&size=' + "7000"
    while True:
        try:
            response = requests.get(url, timeout=TIMEOUT)
            break
        except SSLError:
            time.sleep(5)
        except requests.exceptions.Timeout:
            time.sleep(5)

    return response

def zenodoJsonParser(response):
    print("Parsing result...")
    json_response = response.json()

    dois_list = []
    count = 1
    for element in json_response["hits"]["hits"]:
        type = element["metadata"]["resource_type"]["type"]
        if type == TYPE:
            if count > NB: break
            dois_list.append("https://doi.org/" + element["doi"])
            count += 1

    print("Retrieved " + str(len(dois_list)) + " DOIs")
    return dois_list

In [4]:
response = zenodoRestRequest()
dois_list = zenodoJsonParser(response)
print(len(dois_list))

REST request to zenodo...
Parsing result...
Retrieved 716 DOIs
716


## 2. Selecting a subset of the DOIs to create a sample

In [5]:
import random

random.seed(10)


samples_dois_list = random.sample(dois_list, 500)

splitedSize = 100
samples_chunks = [samples_dois_list[x:x+splitedSize] for x in range(0, len(samples_dois_list), splitedSize)]

In [6]:
print(len(samples_dois_list))

500


In [7]:
# open file in write mode
with open("../results/zenodo_data/url_list", 'w+') as fp:
    for item in samples_dois_list:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

#open and read the file
with open("../results/zenodo_data/url_list") as f:
    lines = [line.strip() for line in f]
    samples_dois_list = lines

Done


## 3. Initiating the list of metrics to be tested

In [8]:
metrics_collection = []
metrics_collection.append(FAIRMetricsFactory.get_A11(None))
metrics_collection.append(FAIRMetricsFactory.get_F1A(None))
metrics_collection.append(FAIRMetricsFactory.get_F1B(None))
metrics_collection.append(FAIRMetricsFactory.get_F2A(None))
metrics_collection.append(FAIRMetricsFactory.get_F2B(None))
metrics_collection.append(FAIRMetricsFactory.get_I1(None))
metrics_collection.append(FAIRMetricsFactory.get_I2(None))
metrics_collection.append(FAIRMetricsFactory.get_I3(None))
metrics_collection.append(FAIRMetricsFactory.get_R11(None))
metrics_collection.append(FAIRMetricsFactory.get_R12(None))
metrics_collection.append(FAIRMetricsFactory.get_R13(None))

df_columns = ['ID']
for m in metrics_collection:
    #print(m.get_principle_tag())
    df_columns.append(m.get_principle_tag())

In [9]:
exec_time_df = pd.DataFrame(columns=df_columns)

def eval_metrics(web_res):
    metrics_collection = []
    metrics_collection.append(FAIRMetricsFactory.get_A11(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F1A(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F1B(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F2A(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F2B(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_I1(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_I2(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_I3(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_R11(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_R12(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_R13(web_res))
    
    row = {"ID":web_res.get_url()}
    row_time = {"ID":web_res.get_url()}
    for m in metrics_collection:
        ts1 = time.time()
        e = m.evaluate()
        duration = round((time.time() - ts1), 2)
        if e is not None:
            row[m.get_principle_tag()] = e.get_score()
            row_time[m.get_principle_tag()] = duration
    
    return row, row_time


## 4. Launching the list of FAIR tests for each DOI

In [10]:
def mass_eval(samples):
    evals = []
    exec_time = []

    for sample in tqdm(samples):
        wr = WebResource(url=sample)
        row, row_time = eval_metrics(wr)
        evals.append(row)
        exec_time.append(row_time)        
    
    return evals, exec_time

In [ ]:
import logging
logging.getLogger().setLevel(logging.ERROR)

i = 0
for c in tqdm(samples_chunks):
    i += 1
    df = pd.DataFrame()
    df_time = pd.DataFrame()
    
    evals, exec_time = mass_eval(c)
    df = pd.concat([df, pd.DataFrame.from_records(evals)])
    df_time = pd.concat([df_time, pd.DataFrame.from_records(exec_time)])
    
    df.to_csv(OUTPUT_DIR + "FC_results_ZENODO_data_"+str(i)+".csv")
    df_time.to_csv(OUTPUT_DIR + "exec_time_ZENODO_data_"+str(i)+".csv")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zenodo.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
INFO:dc67c1fa-094a-46c4-9ee1-419635a1a174:Checking if the URL uses HTTP protocol
INFO:dc67c1fa-094a-46c4-9ee1-419635a1a174:The resource use HTTP protocol
INFO:1bf66ec9-6e76-4115-a7ed-5527b2ac53be:Checking if the URL is reachable, status code: 200
INFO:1bf66ec9-6e76-4115-a7ed-5527b2ac53be:Status code is OK, meaning the u

INFO:d60ad73b-9082-431f-8546-8d4e838dd5d3:Checking that at least one of the following licence properties is found in metadata:

        schema:license 
        dct:license 
        doap:license 
        dbpedia-owl:license 
        cc:license 
        xhv:license 
        sto:license 
        nie:license 
        
INFO:d60ad73b-9082-431f-8546-8d4e838dd5d3:At least one of the licence property was found in metadata !
INFO:2273bb3c-b099-4aa4-8465-183bf4c6bb9d:Checking that at least one of the following provenance properties is found in metadata:

            prov:wasGeneratedBy 
            prov:wasDerivedFrom 
            prov:wasAttributedTo 
            prov:used 
            prov:wasInformedBy 
            prov:wasAssociatedWith
            prov:startedAtTime 
            prov:endedAtTime
            dct:hasVersion 
            dct:creator 
            dct:contributor 
            pav:hasVersion 
            pav:hasCurrentVersion 
            pav:createdBy 
            pav:authoredBy 

INFO:5c29baf2-d8bd-49c0-8c0c-08b662144688:Checking if data is structured, looking for at least one RDF triple...
INFO:5c29baf2-d8bd-49c0-8c0c-08b662144688:41 RDF triples were found, thus data is in a well structured graph format
INFO:a2d13828-3f88-49d5-8ff7-2890be5b92db:Strong evaluation:
INFO:a2d13828-3f88-49d5-8ff7-2890be5b92db:Checking if all classes used in RDF are known in OLS, LOV, or BioPortal
INFO:a2d13828-3f88-49d5-8ff7-2890be5b92db:All classes found in those ontology registries
INFO:a2d13828-3f88-49d5-8ff7-2890be5b92db:Checking if all properties used in RDF are known in OLS, LOV, or BioPortal
INFO:a2d13828-3f88-49d5-8ff7-2890be5b92db:All properties found in those ontology registries
INFO:a2d13828-3f88-49d5-8ff7-2890be5b92db:All classes and properties are known in major ontology registries
INFO:77eb3173-629f-483f-8ef1-d2dec596b939:Checking if data is structured, looking for at least one RDF triple...
INFO:77eb3173-629f-483f-8ef1-d2dec596b939:41 RDF triples were found, thus dat

...

INFO:d7f0197b-a1be-46b2-8b78-33ab0b59044f:None of the provenance property were found in metadata !
INFO:878ba1d3-d01f-4fa2-92ca-276e00bf7891:Strong evaluation:
INFO:878ba1d3-d01f-4fa2-92ca-276e00bf7891:Checking if all classes used in RDF are known in OLS, LOV, or BioPortal
INFO:878ba1d3-d01f-4fa2-92ca-276e00bf7891:All classes found in those ontology registries
INFO:878ba1d3-d01f-4fa2-92ca-276e00bf7891:Checking if all properties used in RDF are known in OLS, LOV, or BioPortal
INFO:878ba1d3-d01f-4fa2-92ca-276e00bf7891:All properties found in those ontology registries
INFO:878ba1d3-d01f-4fa2-92ca-276e00bf7891:All classes and properties are known in major ontology registries
/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

INFO:76e274b4-3be4-4647-a83c-16781829fa35:All classes found in those ontology registries
INFO:76e274b4-3be4-4647-a83c-16781829fa35:Checking if all properties used in RDF are known in OLS, LOV, or BioPortal
INFO:76e274b4-3be4-4647-a83c-16781829fa35:All properties found in those ontology registries
INFO:76e274b4-3be4-4647-a83c-16781829fa35:All classes and properties are known in major ontology registries
INFO:287ad64f-a4b2-4362-8635-2dc0d1b5b114:Checking that at least 3 different URL authorities are used in the URIs of RDF metadata
INFO:287ad64f-a4b2-4362-8635-2dc0d1b5b114:At least 3 different domains were found in metadata (6)
INFO:856268d4-b9bb-4ac0-898a-9d44e5f56248:Checking that at least one of the following licence properties is found in metadata:

        schema:license 
        dct:license 
        doap:license 
        dbpedia-owl:license 
        cc:license 
        xhv:license 
        sto:license 
        nie:license 
        
INFO:856268d4-b9bb-4ac0-898a-9d44e5f56248:At least

INFO:1f4da3eb-8a59-4540-a777-f070bd96751d:Checking if the URL uses HTTP protocol
INFO:1f4da3eb-8a59-4540-a777-f070bd96751d:The resource use HTTP protocol
INFO:a2f9d695-ee87-4cd2-90e0-a295ec7dfa9f:Checking if the URL is reachable, status code: 200
INFO:a2f9d695-ee87-4cd2-90e0-a295ec7dfa9f:Status code is OK, meaning the url is Unique.
INFO:8dfb565c-fe31-4123-aba8-90611fa90070:Strong evaluation:
INFO:8dfb565c-fe31-4123-aba8-90611fa90070:Checking if there is either schema:identifier or dct:identifier property in metadata
INFO:8dfb565c-fe31-4123-aba8-90611fa90070:Found at least one of those property in metadata
INFO:96e5c836-cda1-428d-a4e4-fc592919cc79:Checking if data is structured, looking for at least one RDF triple...
INFO:96e5c836-cda1-428d-a4e4-fc592919cc79:33 RDF triples were found, thus data is in a well structured graph format
INFO:c2c93314-4026-40bf-b1ec-926a1d5f005b:Strong evaluation:
INFO:c2c93314-4026-40bf-b1ec-926a1d5f005b:Checking if all classes used in RDF are known in OLS, 

INFO:5c74a4c6-6c97-4478-9fdd-15f21d7a08c4:None of the provenance property were found in metadata !
INFO:dc1c71cc-5790-4f41-bb7b-78a617a306ef:Strong evaluation:
INFO:dc1c71cc-5790-4f41-bb7b-78a617a306ef:Checking if all classes used in RDF are known in OLS, LOV, or BioPortal
INFO:dc1c71cc-5790-4f41-bb7b-78a617a306ef:All classes found in those ontology registries
INFO:dc1c71cc-5790-4f41-bb7b-78a617a306ef:Checking if all properties used in RDF are known in OLS, LOV, or BioPortal
INFO:dc1c71cc-5790-4f41-bb7b-78a617a306ef:All properties found in those ontology registries
INFO:dc1c71cc-5790-4f41-bb7b-78a617a306ef:All classes and properties are known in major ontology registries
/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

INFO:283843c1-32c8-434f-bf9d-a861cbbdb99f:All classes found in those ontology registries
INFO:283843c1-32c8-434f-bf9d-a861cbbdb99f:Checking if all properties used in RDF are known in OLS, LOV, or BioPortal
INFO:283843c1-32c8-434f-bf9d-a861cbbdb99f:All properties found in those ontology registries
INFO:283843c1-32c8-434f-bf9d-a861cbbdb99f:All classes and properties are known in major ontology registries
INFO:a35cfbc0-2801-4a79-9dcb-0274af20f1bc:Checking that at least 3 different URL authorities are used in the URIs of RDF metadata
INFO:a35cfbc0-2801-4a79-9dcb-0274af20f1bc:At least 3 different domains were found in metadata (5)
INFO:9a4d47e7-8e61-497a-9cff-19305bc3a1a8:Checking that at least one of the following licence properties is found in metadata:

        schema:license 
        dct:license 
        doap:license 
        dbpedia-owl:license 
        cc:license 
        xhv:license 
        sto:license 
        nie:license 
        
INFO:9a4d47e7-8e61-497a-9cff-19305bc3a1a8:At least

In [ ]:
len(df)

In [ ]:
df_time

In [ ]:
import glob

all_files = glob.glob(OUTPUT_DIR + "FC_results_ZENODO_data_*.csv")

li = []

for filename in tqdm(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
from upsetplot import generate_counts, generate_samples, UpSet,plot
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200 

## 5. Generating the UpSetPlot

In [ ]:
from upsetplot import from_contents

A11 = df[((df["A1.1"].astype(int) > 0))]["ID"]

F1A = df[((df["F1A"].astype(int) > 0))]["ID"]
F1B = df[((df["F1B"].astype(int) > 0))]["ID"]
F2A = df[((df["F2A"].astype(int) > 0))]["ID"]
F2B = df[((df["F2B"].astype(int) > 0))]["ID"]

I1 = df[((df["I1"].astype(int) > 0))]["ID"]
I2 = df[((df["I2"].astype(int) > 0))]["ID"]
I3 = df[((df["I3"].astype(int) > 0))]["ID"]

R11 = df[((df["R1.1"].astype(int) > 0))]["ID"]
R12 = df[((df["R1.2"].astype(int) > 0))]["ID"]
R13 = df[((df["R1.3"].astype(int) > 0))]["ID"]

df_upset = from_contents({
    'A11': A11,
    'F1A': F1A, 
    'F1B': F1B, 
    'F2A': F2A, 
    'F2B': F2B, 
    'I1': I1, 
    'I2': I2, 
    'I3': I3, 
    'R11': R11, 
    'R12': R12,
    'R13': R13,
})
df_upset

In [ ]:
upset = UpSet(df_upset, subset_size='count', show_counts=True, sort_categories_by=None, orientation='horizontal')

#upset.style_subsets(present=["with_exome", "with_images"],
#                    facecolor="red")

params = {'legend.fontsize': 8}
with plt.rc_context(params):
    upset.plot()
plt.suptitle("Datasets from Zenodo")
plt.savefig(OUTPUT_DIR + 'zenodo_all_data.png', format="png")
plt.show()